# Imports

In [ ]:
# Installing prophet and arima

# %pip install Prophet
# %pip install pmdarima

In [ ]:
# Imports

import pandas as pd
import yfinance as yf
from scipy.stats import norm
from matplotlib import pyplot as plt
import sklearn
from pathlib import Path
import csv
from prophet import Prophet

import numpy as np
import hvplot.pandas
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from pmdarima.arima.utils import ndiffs





from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose 
from pmdarima import auto_arima 
from pandas import DataFrame
from math import sqrt
import math
from statsmodels.graphics.tsaplots import plot_predict
from sklearn.metrics import mean_squared_error, mean_absolute_error



# FETCHING STOCK DATA (YFinance)

### ~ Indicate Ticker ~

In [ ]:
# Indicate Ticker to pull from Yahoo Finance 
ticker = yf.Ticker("MSFT")

### ~ Indicate data length ~

In [ ]:
# Indicate how far back you would like to test
years = 1

In [ ]:
# Define start and end date
daterange = 365*years

today = pd.Timestamp.today(tz="America/New_York")

start = today + pd.Timedelta(days=-1-daterange)
start = start.strftime('%Y-%m-%d')

end = today
end = end.strftime('%Y-%m-%d')

print(f'Start date: {start}')
print(f'End date: {end}')


In [ ]:
# Show historical data of specified ticker and date range
stock_df = ticker.history(start=start, end=end)
stock_df

In [ ]:
# Create dataframe with date and close price only
stock_df.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
stock_df

In [ ]:
# Plot Ticker historical closing price
stock_df.hvplot(title='Plot for '+str(ticker)+'')

# FBPROPHET MODEL

In [ ]:
# Creating dataframe for fbprophet
stock_df_prophet = stock_df.copy()

# Reset Index, Rename Date/Close columns to 'ds' and 'y' to apply Prophet model
stock_df_prophet.reset_index(inplace=True)
stock_df_prophet.rename(columns={'Date':'ds', 'Close':'y'}, inplace=True)
stock_df_prophet

In [ ]:
# Call the Prophet function, store as an object
model = Prophet(interval_width = 0.95)
model
# Fit the model
model.fit(stock_df_prophet)


### ~ Indicate Projection Period ~

In [ ]:
# Indicate how far out you want to produce predictions
days = 30

In [ ]:
# Create a future dataframe to hold predictions
future_trends = model.make_future_dataframe(periods=days, freq='D')

# View the first and last five rows of the future dataframe
future_trends.head()


In [ ]:
# Make the predictions for the trend data using the future_trends DataFrame
forecast_trends = model.predict(future_trends)

forecast_trends.head()

In [ ]:
# Plot the Prophet predictions for the forecast_trends data
model.plot(forecast_trends)

In [ ]:
# Set the `datetime` index of the forecast data.
forecast_trends = forecast_trends.set_index(['ds'])
stock_df_prophet = stock_df_prophet.set_index(['ds'])
forecast_trends.head()

In [ ]:
forecast_trends.tail()

In [ ]:
# From the `forecast_trends` DataFrame, use hvPlot to visualize
forecast_plot = forecast_trends[['yhat', 'yhat_lower', 'yhat_upper']].hvplot()

actual_plot = stock_df_prophet[["y"]].hvplot(title='Forecast for '+str(ticker)+'')

comparative_plot = forecast_plot * actual_plot
comparative_plot

# FBPROPHET - Accuracy score (mean absolute percentage error)

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:
# Show historical data of specified ticker and date range
stock_df_prophet_test = ticker.history(start='2021-06-03', end='2022-06-03')
stock_df_prophet_test.head()

In [ ]:
# Create dataframe with date and close price only
stock_df_prophet_test.drop(['Open', 'High', 'Low', 'Volume', 'Dividends', 'Stock Splits'], inplace=True, axis=1)
stock_df_prophet_test.head()

In [ ]:
# Reset Index, Rename Date/Close columns to 'ds' and 'y' to apply Prophet model
stock_df_prophet_test.reset_index(inplace=True)
stock_df_prophet_test.rename(columns={'Date':'ds', 'Close':'y'}, inplace=True)
stock_df_prophet_test.head()

In [ ]:
# Call the Prophet function, store as an object
model_test =Prophet(interval_width = 0.95)
model_test
# Fit the model
model_test.fit(stock_df_prophet_test)

In [ ]:
# Create a future dataframe to hold predictions
future_trends_test = model_test.make_future_dataframe(periods=7, freq='D')

# View the first and last five rows of the future dataframe
future_trends_test.tail(10)

In [ ]:
forecast_trends_test = model_test.predict(future_trends_test)

In [ ]:
forecast_trends_test.tail(10)

In [ ]:
forecast_trends_test = forecast_trends_test.set_index(['ds'])
stock_df_prophet_test = stock_df_prophet_test.set_index(['ds'])


In [ ]:
test_data = stock_df_prophet['y'].loc['2022-06-03':'2022-06-09']
prediction = forecast_trends_test['yhat'].loc[['2022-06-03','2022-06-06','2022-06-07','2022-06-08','2022-06-09']]

In [ ]:
test_data

In [ ]:
prediction

In [ ]:
def evaluation(test_input, test_data, predictions):
        r_squared = test_input(test_data, predictions)
        
        return print("Mean Absolute Percentage Score:", r_squared * 100)

In [ ]:
# mean_absolute_percentage_error
evaluation(mean_absolute_percentage_error, test_data, prediction)

# ARIMA MODEL (p, d, q parameters)

In [ ]:
# creating dataframe
stock_df_arima = stock_df
stock_df_arima

### Augumented Dickey Fuller (ADF) test to check if the price series is stationary

In [ ]:
# null hypothesis of the ADF test is that the time series is non-stationary
# if the p-value > 0.05 (meaning it's non-stationary) we'll need to find the order of differencing (d parameter)

In [ ]:
# Check if price series is stationary

result = adfuller(stock_df_arima.Close.dropna())
print(f"ADF Statistic: {result[0]}")
print(f"p-value: {result[1]}")

### (1) d - number of differencing required to make the time series stationary

In [ ]:
# we are looking for d value that has values on ACF plot closest to 0

In [ ]:
# Not Differentiated (d = 0)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(stock_df_arima.Close)
ax1.set_title("Original")
plot_acf(stock_df_arima.Close, ax=ax2);

In [ ]:
# Differentiated Once (d = 1)

differentiated_once = stock_df_arima.Close.diff().dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(differentiated_once)
ax1.set_title("Difference once")
plot_acf(differentiated_once, ax=ax2);

In [ ]:
# Differentiated twice (d = 2)

differentiated_twice = stock_df_arima.Close.diff().diff().dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(differentiated_twice)
ax1.set_title("Difference twice")
plot_acf(differentiated_twice, ax=ax2);

In [ ]:
# Differentiated twice (d = 3)

differentiated_thrice = stock_df_arima.Close.diff().diff().diff().dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(differentiated_twice)
ax1.set_title("Difference thrice")
plot_acf(differentiated_thrice, ax=ax2);

In [ ]:
# Alternatively, can AUTOCALCULATE d value via pmdarima package to confirm manual calculation we did above using ACF plots

d_value = ndiffs(stock_df_arima.Close, test="adf")
print(f"d-value: {d_value}")

### (2) p - the order of the AR term

### ~ indicate dataframe with optimal differentiation (d value) from step 1 ~ 

In [ ]:
# input datafram with optimal differentiation (d value)
d_value_df = differentiated_once

In [ ]:
# p is the order of the Auto Regressive (AR) term. It refers to the number of lags to be used as predictors. 
# We can find out the required number of AR terms by inspecting the Partial Autocorrelation (PACF) plot.
# The partial autocorrelation represents the correlation between the series and its lags. 

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(d_value_df)
ax1.set_title("d = [?]")
plot_pacf(d_value_df, ax=ax2);

In [ ]:
# p value =  lag # that is above (?or below) the significance line 

### (3) q -the order of the MA term

In [ ]:
# q is the order of the Moving Average (MA) term. It refers to the number of lagged forecast errors that should go into the ARIMA Model.
# We can look at the ACF plot for the number of MA terms.

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(d_value_df)
ax1.set_title("d = [?]")
ax2.set_ylim(0, 1)
plot_acf(d_value_df, ax=ax2);

In [ ]:
# q value =  order # that is above (?or below) the significance line ???

### ~ Indicate p, d, q values ~

In [ ]:
p = 0
d = 1
q = 0

## Converting DF into pct_change

In [ ]:
# calculating pct_change - use pct_change data to apply to model
stock_df_arima_pctchange = stock_df_arima.pct_change()
stock_df_arima_pctchange.rename(columns={'Close':'pct change'}, inplace=True)
stock_df_arima_pctchange

### Split Train Test Data

### ~ indicate test data length of days from today ~ 

In [ ]:
days_test_data = 30

In [ ]:
# Test vs Training data split

# df_log = np.log(stock_df_arima)
train_data, test_data = stock_df_arima_pctchange[3:int(len(stock_df_arima_pctchange)-days_test_data)], stock_df_arima_pctchange[int(len(stock_df_arima_pctchange)-days_test_data):]

train_data_close, test_data_close = stock_df_arima[3:int(len(stock_df_arima_pctchange)-days_test_data)], stock_df_arima[int(len(stock_df_arima)-days_test_data):]
# need this variable for combing dataframes later on

plt.figure(figsize=(10,6))
plt.grid(True)
plt.xlabel('Dates')
plt.ylabel('pct change')
plt.plot(stock_df_arima_pctchange, 'green', label='Train data')
plt.plot(test_data, 'blue', label='Test data')
plt.legend()

In [ ]:
train_data.head()

### Apply TRAINING Data (pct_change) to ARIMA Model

In [ ]:
# ARIMA Model
#stock_df_arima.index = pd.DatetimeIndex(stock_df_arima.index).to_period('D')

model = ARIMA(train_data, order=(p, d, q))
result = model.fit()
result.summary()

In [ ]:
# plot residual erros --???
residuals = pd.DataFrame(result.resid)
residuals.plot()
residuals.plot(kind='kde')
print(residuals.describe())

In [ ]:
# Forecasting (indicate how many days out)

forecast = result.get_forecast(days_test_data)

fc = result.forecast(days_test_data)
fc.head()

In [ ]:
ci = forecast.conf_int()

ci.head()

In [ ]:
# combine fc and ci
train_data_fc_ci = pd.concat([fc, ci], axis=1, join="inner")
train_data_fc_ci.head()

In [ ]:
test_data_close.reset_index().head()

In [ ]:
df = test_data_close.index.tolist()
df_2 = test_data_close['Close'].values.tolist()

In [ ]:
train_data_fc_ci['value'] = df_2
train_data_fc_ci['date'] = df
train_data_fc_ci.head()

In [ ]:
train_data_fc_ci = train_data_fc_ci.set_index('date')

In [ ]:
# Convert pct_change back to closing prices using formula:
# (1+pctchange) * prev closing price 
train_data_fc_ci['test data close'] = (train_data_fc_ci['value'].shift(1)  * (1 + train_data_fc_ci['predicted_mean']))
train_data_fc_ci['Lower CI'] = (train_data_fc_ci['value'].shift(1)  * (1 + train_data_fc_ci['lower pct change']))
train_data_fc_ci['Upper CI'] = (train_data_fc_ci['value'].shift(1)  * (1 + train_data_fc_ci['upper pct change']))
train_data_fc_ci.head()

In [ ]:
train_data_fc_ci.plot()

In [ ]:
test_data_close.plot()

In [ ]:
# Actual vs Projected Price of Testing data
plt.plot(train_data_fc_ci['value'])
plt.plot(train_data_fc_ci['test data close'])

In [ ]:
# combine graphs
plt.plot(train_data_close, label="Close", color="blue")
plt.plot(train_data_fc_ci['value'], color="blue")
plt.plot(train_data_fc_ci['test data close'], color="red")
plt.plot(train_data_fc_ci['Lower CI'], color="grey")
plt.plot(train_data_fc_ci['Upper CI'], color="grey")


### adding dates to forecasted price

In [ ]:
# date autogeneration ---> but needs only weekdays ########################
date_testdata = pd.date_range(end=today, freq='D', periods=30)
date_testdata

In [ ]:
# comibning date with forecasted and ci df ############## error

stock_df_arima_traindata_date = pd.concat(train_data_fc_ci, date_testdata), axis=1, ignore_index=True)


### Convert pct_change back to CLose price

In [ ]:
# Convert pct_change back to closing prices using formula:
# (1+pctchange) * prev closing price 
stock_df_arima['test data close'] = (stock_df_arima['Close'].shift(1)  * (1 + stock_df_arima['test data pct_change']))
stock_df_arima

### Forecasting

In [ ]:
# ARIMA Model
#stock_df_arima.index = pd.DatetimeIndex(stock_df_arima.index).to_period('D')

model = ARIMA(stock_df_pct.Close, order=(p, d, q))
result = model.fit()
result.summary()

In [ ]:
# Forecasting (indicate how many days out)

forecast = result.get_forecast(51)

fc = result.forecast(51)
fc.head()

In [ ]:
ci = forecast.conf_int()

ci.head()

In [ ]:
# combine fc and ci
fc_df_arima = pd.concat([fc, ci], axis=1, join="inner")
fc_df_arima.head()

In [ ]:
# Actual vs Fitted plot ???? [ delete]

plot_predict(
   result, 
   start=1,
   end=60,
  dynamic=False);

In [ ]:
forecast_plot_arima_ci = ci[['lower Close', 'upper Close']].hvplot()


actual_plot_arima = stock_df_arima[["Close"]].hvplot(title='Forecast for '+str(ticker)+'')

comparative_plot_arima = forecast_plot_arima_ci * actual_plot_arima
comparative_plot_arima

### Make a panda series / Graph

In [ ]:
fc = pd.Series(fc, index=test_data[:step].index)


In [ ]:
lower = pd.Series(ci[:, 0], index=test_data[:step].index)
# upper = pd.Series(ci[:, 1], index=test_data[:step].index)

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(test[:step], label="actual")
plt.plot(fc, label="forecast")
plt.fill_between(lower.index, lower, upper, color="k", alpha=0.1)
plt.title("Forecast vs Actual")
plt.legend(loc="upper left")

### Auto ARIMA (shortcut for figuring out optimal q, d, p)

In [ ]:
model = auto_arima(
    stock_df_arima.Close,
    start_p=1,
    start_q=1,
    test="adf",
    max_p=6,
    max_q=6,
    m=1,  # frequency of series
    d=None,  # determine 'd'
    seasonal=False,  # no seasonality
    trace=True,
    stepwise=True,
)



# ARIMA - Accuracy score (mean absolute percentage error)

In [ ]:
# forecasted closing price
a = train_data_fc_ci['test data close']
a.dropna()
# actual closing price
b = train_data_fc_ci['value']
b.dropna()

In [ ]:
mse = mean_squared_error(b, a)
print('MSE: '+str(mse))
mae = mean_absolute_error(b, a)
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(b, a))
print('RMSE: '+str(rmse))
mape = np.mean(np.abs(a - b)/np.abs(b))
print('MAPE: '+str(mape))

# FBPROPHET vs ARIMA - analysis, graphs